<a href="https://colab.research.google.com/github/esmeii/automation_with_python/blob/main/weatherAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import datetime

village_weather_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?"

service_key = "동네예보조회 인증키"
base_data = datetime.datetime.today().strftime("%Y%m%d")
base_time = "0800"
nx = "59"
ny = "126"

payload = "serviceKey="+service_key+"&"+\
  "dataType=json"+"&"+\
  "base_time"+base_time+"&"+\
  "base_date"+base_date+"&"+\
  "nx"+nx+"&"+\
  "ny"+ny

pty_code = {"0":"없음", "1":"비","2":"비/눈","3":"눈","4":"소나기","5":"빗방울","6":"빗방울/눈날림","7":"눈"}
data = dict()
data['data'] = base_data
weather = dict()

#값 요청
res = requests.get(vilage_weather_url + payload)
try:
  items = res.json().get('response').get('body').get('items')
  for item in items['item']:
    #기온
    if item['category'] =='T3H':
      weather['tmp'] = item['fcstValue']
    #강수상태
    if item['category'] =='PTY':
      weather['code'] = item['fcstValue']
      weather['state']= pty_code[item['fcstValue']]
except:
      print("날씨 정보 가져오기 실패: ", res.text)
data['weather'] = weather
print(data['weather'])

In [ ]:
#미세먼지 정보 얻기 및 정제

def get_pm10_state(pm10_value):
  if pm10_value < 30:
    pm10_state = "좋음"
  elif pm10_value <80:
    pm10_state = "보통"
  elif pm10_value <150:
    pm10_state = "나쁨"
  else:
    pm10_state = "매우 나쁨"
  return pm10_state
def get_pm25_state(pm25_value):
  if pm25_value < 30:
    pm25_state = "좋음"
  elif pm25_value <80:
    pm25_state = "보통"
  elif pm25_value <150:
    pm25_state = "나쁨"
  else:
    pm25_state = "매우 나쁨"
  return pm25_state

#미세먼지 데이터 요청
dust_url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty?"
service_key = "환경공단 에어코리아 대기오염 정보의 인증키"

payload = "serviceKey="+service_key+"&"+\
  "dataType=json"+"&"+\
  "sidoName=서울"+"&"+\
  "var=1.0"

#pm10 pm2.5 수치 가져오기
res = requests.get(dust_url+payload)
result = res.json()
dust = dict()
if(res.status_code == 200)&(result['response']['header']['resultCode'] == '00'):
  dust['PM10'] = {'value' : int(result['response']['body']['items'][0]['pm10Value'])}
  dust['PM2.5'] = {'value' : int(result['response']['body']['items'][0]['pm2.5Value'])}

  #PM10 미세먼지 30 80 150
  pm10_value = dust.get('PM10').get('value')
  pm10_state = get_pm10+state(pm10_value)
  #PM2.5 초미세먼지 15 35 75
  pm25_value = dust.get('PM2.5').get('value')
  pm25_state = get_pm25+state(pm25_value)
else:
  print("미세먼지 가져오기 실패 ",result['response']['header']['resultMsg'])

data['dust'] = dust
print(data['dust'])